In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
#from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd
import numpy as np
import re

In [2]:
Data = pd.read_csv('data.tsv', sep='\t', error_bad_lines=False)

/var/folders/tr/3_k_czmn2552n9rftmys1pwh0000gn/T/ipykernel_47000/806793109.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Data = pd.read_csv('data.tsv', sep='\t', error_bad_lines=False)
b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/var/folders/tr/3_k_czmn2552n9rftmys1pwh0000gn/T/ipykernel_47000/806793109.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  Data = pd.read_csv('data.tsv', sep='\t', error_bad_lines=False)


In [3]:
Data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [4]:
Data.shape

(363192, 6)

In [5]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363192 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363192 non-null  object 
 1   qid1          363192 non-null  object 
 2   qid2          363185 non-null  float64
 3   question1     363181 non-null  object 
 4   question2     363180 non-null  object 
 5   is_duplicate  363180 non-null  float64
dtypes: float64(2), object(4)
memory usage: 16.6+ MB


In [6]:
# remove duplicates
Data.drop_duplicates(inplace=True)

In [7]:
Data.shape

(363191, 6)

In [8]:
# fillter the first 100 'question1' with 'is_duplicate' == 1.0
Q1 = Data[Data['is_duplicate'] == 1.0].head(100)
# keep only the 'question1' column
Q1 = Q1['question1']

In [9]:
Q2= Data['question2']
# test Q2 as the first 1000 'question2'
#Q2 = Data['question2'].head(1000)
Q2 = Q2.astype(str) # make sure the type is string


## Preprocessing

In [10]:
#Process the review column line by line to do text preprocessing
def process_review(review):
    # remove the punctuations and numbers
    review = re.sub(r'[^A-Za-z]+', ' ', review)
    # convert the review to lower case
    review = review.lower()
    # remove the stopwords
    stop_words = set(stopwords.words('english'))
    # tokenize the words
    word_tokens = word_tokenize(review)
    filtered_review = [w for w in word_tokens if not w in stop_words]
    # stem the words
    stemmer = PorterStemmer()
    stemmed_review = [stemmer.stem(w) for w in filtered_review]
    # # lemmatize the words
    # lemmatizer = WordNetLemmatizer()
    # lemmatized_review = [lemmatizer.lemmatize(w) for w in stemmed_review]
    # return the processed review
    return stemmed_review

In [11]:
# process the train and test reviews
Q1 = Q1.apply(process_review)
Q2 = Q2.apply(process_review)

In [12]:
Q1.head()

1                                [control, horni, emot]
3                                            [one, mbb]
7           [best, self, help, book, read, chang, life]
11    [hillari, clinton, polici, toward, india, beco...
13       [best, book, studi, tensor, gener, rel, basic]
Name: question1, dtype: object

In [13]:
Q2.head()

0                 [level, preprat, enough, exam, jlpt]
1                                     [control, horni]
2                     [caus, stool, come, littl, ball]
3                                                [mbb]
4    [would, second, airport, sydney, australia, ne...
Name: question2, dtype: object

## TF-IDF

In [14]:
#build a unique vocabulary from the Q2
vocab = set()
for Q in Q2:
    vocab.update(Q)

len(vocab)

39990

In [15]:
# covert each word into a unique integer
word2int = {}
for i,word in enumerate(vocab):
    word2int[word] = i

# show the first 10 in word2int dictionary
list(word2int.items())[:10]

[('weldon', 0),
 ('havala', 1),
 ('hagrid', 2),
 ('coy', 3),
 ('allus', 4),
 ('apostroph', 5),
 ('vasoconstrict', 6),
 ('ramess', 7),
 ('tambora', 8),
 ('kanha', 9)]

In [16]:
# search by key
word2int['move']

# search by value
def get_key(val):
    for key, value in word2int.items():
        if val == value:
            return key
        
    return "key doesn't exist"

get_key(836)


'xnxx'

In [17]:
# create a term frequency matrix
def count_TF_matrix(Q):
    # initialize a matrix of zeros
    row = len(Q)
    col = len(vocab)
    tf_matrix = np.zeros((row, col), dtype=np.float32)
    # fill the matrix with the frequency of each word
    for i, Q in enumerate(Q):
        len_Q = len(Q)
        for word in Q:
            tf_matrix[i][word2int[word]] += 1/len_Q
    return tf_matrix

In [18]:
TF_matrix_Q2 = count_TF_matrix(Q2)

In [19]:
TF_matrix_Q2.shape

(363191, 39990)

In [20]:
# calculate the IDF 
N = len(TF_matrix_Q2)
DF = np.count_nonzero(TF_matrix_Q2, axis=0)
DF = DF.astype(np.float32)
IDF = np.log(N/DF + 1)

In [21]:
# calculate the TF-IDF matrix
TF_IDF_matrix_Q2 = TF_matrix_Q2 * IDF
TF_IDF_matrix_Q2.shape


: 

: 

In [ ]:
# create a inverted file dictionary, with key as word and value as the (doc_id, TF-IDF) tuple
def create_inverted_file(TF_IDF_matrix):
    inverted_file = {}
    for i, Q in enumerate(Q2):
        for word in Q:
            if word not in inverted_file.keys():
                inverted_file[word] = [(i, TF_IDF_matrix[i][word2int[word]])]
            else:
                inverted_file[word].append((i, TF_IDF_matrix[i][word2int[word]]))
    return inverted_file

In [ ]:
inverted_file_Q2 = create_inverted_file(TF_IDF_matrix_Q2)
inverted_file_Q2

In [ ]:
# test the inverted file
# get the first query in Q1
test_Q = Q1.iloc[3]
test_Q # the ground truth is 11

In [ ]:
# get the top5 similar questions
def get_top5_similar_questions(test_Q):
    # initialize a dictionary to store the similarity score
    similarity_score = {}
    # calculate the similarity score
    for word in test_Q:
        if word in inverted_file_Q2.keys():
            for i, TF_IDF in inverted_file_Q2[word]:
                if i not in similarity_score.keys():
                    similarity_score[i] = TF_IDF
                else:
                    similarity_score[i] += TF_IDF
    # sort the similarity score in descending order
    similarity_score = sorted(similarity_score.items(), key=lambda x: x[1], reverse=True)
    # get the top5 similar questions
    top5_similar_questions = []
    for i in range(5):
        top5_similar_questions.append(similarity_score[i][0])
    return top5_similar_questions

In [ ]:
top_index = get_top5_similar_questions(test_Q)
top_index

In [ ]:
# print sentences acording to the index
for i in top_index:
    print(Q2.iloc[i])
    print('\n')
